In [2]:
### Setup
# Import pandas
import pandas as pd

# Set path to files and save in a variable
schools_file = "Resources/schools_complete.csv"
students_file = "Resources/students_complete.csv"

# Store files in dataframes
schools_df = pd.read_csv(schools_file)
students_df = pd.read_csv(students_file)

# Combine dataframes
df = pd.merge(schools_df, students_df, how = "left", on = ["school_name"])
df = df.rename(columns = {"school_name" : "School Name", "type" : "Type", "size" : "Size", "budget" : "Budget", "student_name" : "Student Name", "gender" : "Gender", "grade" : "Grade", "reading_score" : "Reading Score", "math_score" : "Math Score"})
df.head()

,School ID,School Name,Type,Size,Budget,Student ID,Student Name,Gender,Grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [3]:
### District Summary 

#Count unique Schools
count_unique_schools = len(df["School Name"].unique())
print(count_unique_schools)

#Total students
total_students = df["Student ID"].count()
print(total_students)

#Total budget
list_unique_budgets = df["Budget"].unique()
list_unique_budgets_df = pd.DataFrame(list_unique_budgets)
list_unique_budgets_df.head()

total_budget = list_unique_budgets_df[0].sum()
print(total_budget)

# Average math score
average_math_score = df["Math Score"].mean()
print(average_math_score)

# Average reading score
average_reading_score = df["Reading Score"].mean()
print(average_reading_score)

# Clean up columns 
df = df.dropna(how = 'any')


# % of students who passed math
df = df.astype({"Math Score" : "int64"})
pass_math_list = df.loc[df["Math Score"] >= 70, :]
pass_math_df = pd.DataFrame(pass_math_list)
pass_math_df.head()
count_math_pass = pass_math_df["Student ID"].count()

percent_pass_math = count_math_pass / total_students
print(percent_pass_math)


# % of students who passed reading
df = df.astype({"Reading Score" : "int64"})
pass_reading_list = df.loc[df["Reading Score"] >= 70, :]
pass_reading_df = pd.DataFrame(pass_reading_list)
count_reading_pass = pass_reading_df["Student ID"].count()

percent_pass_reading = count_reading_pass / total_students
print(percent_pass_reading)


# % of students who passed math AND reading
pass_mathreading_list = df.loc[(df["Math Score"] >= 70) & (df["Reading Score"] >= 70), :]
pass_mathreading_df = pd.DataFrame(pass_mathreading_list)
count_mathreading_pass = pass_mathreading_df["Student ID"].count()

percent_pass_mathreading = count_mathreading_pass / total_students
print(percent_pass_mathreading)

# Create summary dataframe
district_summary_df = pd.DataFrame({"Total Unique Schools" : [count_unique_schools],
                                   "Total Students" : [total_students],
                                   "Total Budget" : [total_budget],
                                   "Average Math Score" : [average_math_score],
                                   "Average Reading Score" : [average_reading_score],
                                   "Percent of Students Passing Math" : [percent_pass_math],
                                   "Percent of Students Passing Reading" : [percent_pass_reading],
                                   "Percent of Students Passing Overall" : [percent_pass_mathreading]
                                   })
district_summary_df


15
39170
24649428
78.98537145774827
81.87784018381414
0.749808526933878
0.8580546336482001
0.6517232575950983


,Total Unique Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent of Students Passing Math,Percent of Students Passing Reading,Percent of Students Passing Overall
0,15,39170,24649428,78.985371,81.87784,0.749809,0.858055,0.651723


In [4]:
### School Summary

#Set Up
school_summary = schools_df.set_index(["school_name"])["type"]
school_summary_df = pd.DataFrame(school_summary)

#Total students per school
students_per_school = df.groupby(["School Name"]).count()["Student ID"]


#Total budget per school
budget_per_school = df.groupby(["School Name"]).mean()["Budget"]


#Total budget per student per school
budget_per_student = budget_per_school / students_per_school


#Average math score per school
total_school_math_score = df.groupby(["School Name"]).sum()["Math Score"]
average_school_math_score = total_school_math_score / students_per_school

#Average reading score per school
total_school_reading_score = df.groupby(["School Name"]).sum()["Reading Score"]
average_school_reading_score = total_school_reading_score / students_per_school


# % of students who passed math per school
df["Pass Math"] = df["Math Score"] >= 70
filter_pass_math_df = df.loc[df["Pass Math"] == True, :]
count_pass_math_per_school = filter_pass_math_df.groupby(["School Name"]).count()["Student ID"]
percent_pass_math_per_school = count_pass_math_per_school / students_per_school


# % of students who passed reading per school
df["Pass Reading"] = df["Reading Score"] >= 70
filter_pass_reading_df = df.loc[df["Pass Reading"] == True, :]
count_pass_reading_per_school = filter_pass_reading_df.groupby(["School Name"]).count()["Student ID"]
percent_pass_reading_per_school = count_pass_reading_per_school / students_per_school

# % of students who passed math AND reading per school
df["Pass Math and Reading"] = ((df["Reading Score"] >= 70) & (df["Math Score"] >= 70))
filter_pass_mathreading_df = df.loc[df["Pass Math and Reading"] == True, :]
count_pass_mathreading_per_school = filter_pass_mathreading_df.groupby(["School Name"]).count()["Student ID"]
percent_pass_mathreading_per_school = count_pass_mathreading_per_school / students_per_school

# Create summary dataframe
school_summary_df = school_summary_df.rename(columns = {"type" : "Type"})
school_summary_df["Total Students"] = students_per_school
school_summary_df["Total Budget"] = budget_per_school
school_summary_df["Per Student Budget"] = budget_per_student
school_summary_df["Average Math Score"] = average_school_math_score
school_summary_df["Average Reading Score"] = average_school_reading_score
school_summary_df["% Passing Math"] = percent_pass_math_per_school
school_summary_df["% Passing Reading"] = percent_pass_reading_per_school
school_summary_df["% Passing Overall"] = percent_pass_mathreading_per_school
school_summary_df.head()


/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/1727305227.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  budget_per_school = df.groupby(["School Name"]).mean()["Budget"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/1727305227.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_school_math_score = df.groupby(["School Name"]).sum()["Math Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/1727305227.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default 

,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Shelton High School,Charter,1761,1056600.0,600.0,83.359455,83.725724,0.938671,0.958546,0.898921
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.971390,0.905995


In [5]:
# Highest performing schools by % overall passing
top_schools_df = school_summary_df.sort_values("% Passing Overall", ascending = False)
top_schools_df.head()


,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [6]:
# Lowest performing schools by % overall passing
bottom_schools_df = school_summary_df.sort_values("% Passing Overall")
bottom_schools_df.head()

,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [7]:
# Math scores by grade
average_math_by_grade = df.groupby(["Grade"]).mean()["Math Score"]
average_math_by_grade_df = pd.DataFrame(average_math_by_grade)
average_math_by_grade_df.head()

/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/801792865.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_math_by_grade = df.groupby(["Grade"]).mean()["Math Score"]


,Math Score
Grade,
10th,78.941483
11th,79.083548
12th,78.993164
9th,78.935659


In [8]:
# Reading scores by grade
average_reading_by_grade = df.groupby(["Grade"]).mean()["Reading Score"]
average_reading_by_grade_df = pd.DataFrame(average_reading_by_grade)
average_reading_by_grade_df.head()

/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/3502942779.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_reading_by_grade = df.groupby(["Grade"]).mean()["Reading Score"]


,Reading Score
Grade,
10th,81.874410
11th,81.885714
12th,81.819851
9th,81.914358


In [9]:
# Scores by school spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary_df["Spend Range"] = pd.cut(school_summary_df["Per Student Budget"], spending_bins, labels = labels, include_lowest = True)

spending_math_scores = school_summary_df.groupby(["Spend Range"]).mean()["Average Math Score"]
spending_reading_scores = school_summary_df.groupby(["Spend Range"]).mean()["Average Reading Score"]
spending_passing_math = school_summary_df.groupby(["Spend Range"]).mean()["% Passing Math"]
spending_passing_reading = school_summary_df.groupby(["Spend Range"]).mean()["% Passing Reading"]
overall_passing_spending = school_summary_df.groupby(["Spend Range"]).mean()["% Passing Overall"]

spending_summary_df = pd.DataFrame({"Average Math Score" : [spending_math_scores],
                                   "Average Reading Score" : [spending_reading_scores],
                                   "% Passing Math (Percentage Of Students Who Passed Math)" : [spending_passing_math],
                                   "% Passing Reading (Percentage Of Students Who Passed Reading)" : [spending_passing_reading],
                                   "% Overall Passing (Percentage Of Students Who Passed Math and Reading)" : [overall_passing_spending],
                                   })

spending_summary_df

/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/3505087577.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_math_scores = school_summary_df.groupby(["Spend Range"]).mean()["Average Math Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/3505087577.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spending_reading_scores = school_summary_df.groupby(["Spend Range"]).mean()["Average Reading Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/3505087577.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is dep

,Average Math Score,Average Reading Score,% Passing Math (Percentage Of Students Who Passed Math),% Passing Reading (Percentage Of Students Who Passed Reading),% Overall Passing (Percentage Of Students Who Passed Math and Reading)
0,Spend Range <$585 83.455399 $585-630 ...,Spend Range <$585 83.933814 $585-630 ...,Spend Range <$585 0.934601 $585-630 0...,Spend Range <$585 0.966109 $585-630 0...,Spend Range <$585 0.903695 $585-630 0...


In [10]:
# Scores by School Size

size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], size_bins, labels = size_labels, include_lowest = True)

math_scores_size = school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
reading_scores_size = school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
passing_math_size = school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
passing_reading_size = school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
passing_spending_size = school_summary_df.groupby(["School Size"]).mean()["% Passing Overall"]

size_summary_df = pd.DataFrame({"Average Math Score" : [math_scores_size],
                                   "Average Reading Score" : [reading_scores_size],
                                   "% Passing Math (Percentage Of Students Who Passed Math)" : [passing_math_size],
                                   "% Passing Reading (Percentage Of Students Who Passed Reading)" : [passing_reading_size],
                                   "% Overall Passing (Percentage Of Students Who Passed Math and Reading)" : [passing_spending_size],
                                   })

size_summary_df


/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/211258118.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  math_scores_size = school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/211258118.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  reading_scores_size = school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/211258118.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. I

,Average Math Score,Average Reading Score,% Passing Math (Percentage Of Students Who Passed Math),% Passing Reading (Percentage Of Students Who Passed Reading),% Overall Passing (Percentage Of Students Who Passed Math and Reading)
0,School Size Small (<1000) 83.821598 Me...,School Size Small (<1000) 83.929843 Me...,School Size Small (<1000) 0.935502 Med...,School Size Small (<1000) 0.960994 Med...,School Size Small (<1000) 0.898839 Med...


In [11]:
# Scores by School Type
math_scores_type = school_summary_df.groupby(["Type"]).mean()["Average Math Score"]
reading_scores_type = school_summary_df.groupby(["Type"]).mean()["Average Reading Score"]
passing_math_type = school_summary_df.groupby(["Type"]).mean()["% Passing Math"]
passing_reading_type = school_summary_df.groupby(["Type"]).mean()["% Passing Reading"]
passing_spending_type = school_summary_df.groupby(["Type"]).mean()["% Passing Overall"]

type_summary_df = pd.DataFrame({"Average Math Score" : [math_scores_type],
                                   "Average Reading Score" : [reading_scores_type],
                                   "% Passing Math (Percentage Of Students Who Passed Math)" : [passing_math_type],
                                   "% Passing Reading (Percentage Of Students Who Passed Reading)" : [passing_reading_type],
                                   "% Overall Passing (Percentage Of Students Who Passed Math and Reading)" : [passing_spending_type],
                                   })

type_summary_df

/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/876361566.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  math_scores_type = school_summary_df.groupby(["Type"]).mean()["Average Math Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/876361566.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  reading_scores_type = school_summary_df.groupby(["Type"]).mean()["Average Reading Score"]
/var/folders/9h/j3dl32y94_sg76r_7j5tmq8w0000gn/T/ipykernel_63303/876361566.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future vers

,Average Math Score,Average Reading Score,% Passing Math (Percentage Of Students Who Passed Math),% Passing Reading (Percentage Of Students Who Passed Reading),% Overall Passing (Percentage Of Students Who Passed Math and Reading)
0,Type Charter 83.473852 District 76.9567...,Type Charter 83.896421 District 80.9666...,Type Charter 0.936208 District 0.665485...,Type Charter 0.965865 District 0.807991...,Type Charter 0.904322 District 0.536722...
